## Structured Self-attentive Sentence Embedding

The code is based on [gluon-nlp tutorial](https://gluon-nlp.mxnet.io/examples/sentence_embedding/self_attentive_sentence_embedding.html).

## Import Related Packages

In [4]:
import os
import json
import zipfile
import time
import itertools

import numpy as np
import mxnet as mx
import multiprocessing as mp
import gluonnlp as nlp

from mxnet import gluon, nd, init
from mxnet.gluon import nn, rnn
from mxnet import autograd, gluon, nd
from d2l import try_gpu
import pandas as pd

# iUse sklearn's metric function to evaluate the results of the experiment
from sklearn.metrics import accuracy_score, f1_score

# fixed random number seed
np.random.seed(2333)
mx.random.seed(2333)

## Data pipeline

### Load Dataset

See [dataloader](data_loader.ipynb) for how the training samples are obtained.

In [5]:
data_folder = 'data/imdb/'
file_name = 'train.csv'
file_path = data_folder + file_name
dev = False

# load train file
if dev:
    # load only n rows
    nrows = 5000
    data = pd.read_csv(file_path, nrows=nrows, sep='|')
else:
    # load all
    data = pd.read_csv(file_path, sep='|')

# create a list of review a label paris.
dataset = [[left, right, int(label)] for left, right, label in \
           zip(data['review_text'], data['plot_summary'], data['is_spoiler'])]

# randomly divide one percent from the training set as a verification set.
train_dataset, valid_dataset = nlp.data.train_valid_split(dataset)
len(train_dataset), len(valid_dataset)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 217, saw 9


### Data Processing

The purpose of the following code is to process the raw data so that the processed data can be used for model training and prediction. We will use the `SpacyTokenizer` to split the document into tokens, `ClipSequence` to crop the comments to the specified length, and build a vocabulary based on the word frequency of the training data. Then we attach the [Glove](https://nlp.stanford.edu/pubs/glove.pdf)[2]  pre-trained word vector to the vocabulary and convert each token into the corresponding word index in the vocabulary.
Finally get the standardized training data set and verification data set

In [3]:
# tokenizer takes as input a string and outputs a list of tokens.
tokenizer = nlp.data.SpacyTokenizer('en')

# length_clip takes as input a list and outputs a list with maximum length 300.
length_clip_review = nlp.data.ClipSequence(300)
length_clip_plot = nlp.data.ClipSequence(100)

def preprocess(x):

    # now the first element in tuple is review, second plot and third label
    try:
        left, right, label = x[0], x[1], int(x[2])
        assert(type(left)==type('str') and type(right)==type('str'))
        assert(label==0 or label==1)
    except:
        print(left, right)
        left, right=str(left), str(right)
    # clip the length of review words
    left, right = length_clip_review(tokenizer(left.lower())), \
                  length_clip_plot(tokenizer(right.lower()))
    return left, right, label

def get_length(x):
    return float(len(x[0]))

def preprocess_dataset(dataset):
    start = time.time()

    with mp.Pool() as pool:
        # Each sample is processed in an asynchronous manner.
        dataset = gluon.data.ArrayDataset(pool.map(preprocess, dataset))
        lengths = gluon.data.ArrayDataset(pool.map(get_length, dataset))
    end = time.time()

    print('Done! Tokenizing Time={:.2f}s, #Sentences={}'.format(end - start, len(dataset)))
    return dataset, lengths

# Preprocess the dataset
train_dataset, train_data_lengths = preprocess_dataset(train_dataset)
valid_dataset, valid_data_lengths = preprocess_dataset(valid_dataset)

The women are hot, things end there Yet another erotic thriller involving a web of lies and intrigue, this time around though there are no decent sex scenes to redeem it. Our protagonist must be one of the most lifeless human beings ever captured on celluloid. He has an incredibly good-looking wife that he never seems to look at or want to be with, but apparently he doesn't want to cheat on her with his co-worker either because damn it, the guy just really can't stand beautiful women. Back to incoherent rants about his work. The nudity is kept to an absolute minimum, which is pretty much the last thing you need in this sleazy kind of flick. Tawny Kitaen (or possibly her body double) has one brief nude scene, Shannon Whirry keeps her clothes on for the entire movie. That's just weird, is anyone really supposed to watch this for the gifted actors or the subtle writing? The ending is pretty inventive, I'll give it that. nan
Go rent a playboy video I heard Tinto Brass did eroticism, fine. 

Process ForkPoolWorker-2:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-11:
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/steve

  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/site-packages/gluonnlp/data/transforms.py", line 324, in __call__
    return [tok.text for tok in self._nlp(sample)]
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/site-packages/spacy/language.py", line 382, in __call__
    doc = self.make_doc(text)
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/steven/miniconda3/envs/dl/lib/python3.7/multiprocessi

KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# create vocab
train_seqs = [sample[0]+sample[1] for sample in train_dataset]
counter = nlp.data.count_tokens(list(itertools.chain.from_iterable(train_seqs)))

vocab = nlp.Vocab(counter, max_size=20000)

# load pre-trained embedding, Glove
embedding_weights = nlp.embedding.GloVe(source='glove.twitter.27B.200d')
vocab.set_embedding(embedding_weights)
print(vocab)

# NOTE: to use the same encoder, we need to ensure that two inputs are of the same length
# this is achieved by manual padding
def token_to_idx(x):
    return vocab[x[0]], vocab[x[1]], x[2]

# A token index or a list of token indices is returned according to the vocabulary.
with mp.Pool() as pool:
    train_dataset = pool.map(token_to_idx, train_dataset)
    valid_dataset = pool.map(token_to_idx, valid_dataset)

In [ ]:
idx = vocab.embedding.token_to_idx['xmen']
idx, vocab.embedding.idx_to_vec[idx]

## Bucketing and DataLoader
Since each sentence may have a different length, we need to use `Pad` to fill the sentences in a minibatch to equal lengths so that the data can be quickly tensored on the GPU. At the same time, we need to use `Stack` to stack the category tags of a batch of data. For convenience, we use `Tuple` to combine `Pad` and `Stack`.

In order to make the length of the sentence pad in each minibatch as small as possible, we should make the sentences with similar lengths in a batch as much as possible. In light of this, we consider constructing a sampler using `FixedBucketSampler`, which defines how the samples in a dataset will be iterated in a more economic way.

Finally, we use `DataLoader` to build a data loader for the training. dataset and validation dataset. The training dataset requires FixedBucketSampler, but the validation dataset doesn't require the sampler.

In [ ]:
batch_size = 64
bucket_num = 10
bucket_ratio = 0.5


def get_dataloader():
    # Construct the DataLoader Pad data, stack label and lengths
    batchify_fn = nlp.data.batchify.Tuple(nlp.data.batchify.Pad(axis=0), \
                                          nlp.data.batchify.Pad(axis=0),
                                          nlp.data.batchify.Stack())

    # n this example, we use a FixedBucketSampler,
    # which assigns each data sample to a fixed bucket based on its length.
    batch_sampler = nlp.data.sampler.FixedBucketSampler(
        train_data_lengths,
        batch_size=batch_size,
        num_buckets=bucket_num,
        ratio=bucket_ratio,
        shuffle=True)
    print(batch_sampler.stats())

    # train_dataloader
    train_dataloader = gluon.data.DataLoader(
        dataset=train_dataset,
        batch_sampler=batch_sampler,
        batchify_fn=batchify_fn)
    # valid_dataloader
    valid_dataloader = gluon.data.DataLoader(
        dataset=valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        batchify_fn=batchify_fn)
    return train_dataloader, valid_dataloader

train_dataloader, valid_dataloader = get_dataloader()

In [ ]:
# experiement one the two dataloaders
# for left, right, label in train_dataloader:
    # print(left.shape, right.shape, label.shape)

In [ ]:
class WeightedSoftmaxCE(nn.HybridBlock):
    def __init__(self, sparse_label=True, from_logits=False,  **kwargs):
        super(WeightedSoftmaxCE, self).__init__(**kwargs)
        with self.name_scope():
            self.sparse_label = sparse_label
            self.from_logits = from_logits

    def hybrid_forward(self, F, pred, label, class_weight, depth=None):
        if self.sparse_label:
            label = F.reshape(label, shape=(-1, ))
            label = F.one_hot(label, depth)
        if not self.from_logits:
            pred = F.log_softmax(pred, -1)

        weight_label = F.broadcast_mul(label, class_weight)
        loss = -F.sum(pred * weight_label, axis=-1)

        # return F.mean(loss, axis=0, exclude=True)
        return loss

In [ ]:
class AlignAttention(nn.Block):
    def __init__(self, **kwargs):
        super(AlignAttention, self).__init__(**kwargs)
    
    def forward(self, inp_left, inp_right):
        # input dimension is of (batch_size, seq_len, embed_size)
        # att dimension is of (batch_size, seq_len_left, seq_len_right)
        att = nd.batch_dot(inp_left, nd.transpose(inp_right, axes = (0, 2, 1)))
        # inp_left_dot dimention is of (batch_size, seq_left, embed_size)
        inp_left_dot = nd.batch_dot(nd.softmax(att, axis=-1), inp_right)
        # inp_right_dot dimension is of (batch_size, seq_right, embed_size)
        inp_right_dot = nd.batch_dot(nd.softmax(nd.transpose(att, axes=(0, 2, 1)), axis=-1), inp_left)
        # concat original (lstm output, dot multiplier, substraction, elementwise product)
        # therefore, the real size is (batch_size, seq_len_left/right, embed_size*4)
        aug_left = nd.concat(inp_left, inp_left_dot, inp_left-inp_left_dot, inp_left*inp_left_dot, dim=-1)
        aug_right = nd.concat(inp_right, inp_right_dot, inp_right-inp_right_dot, inp_right*inp_right_dot, dim=-1)
        return aug_left, aug_right

In [ ]:
class EnhancedSeqInfer(nn.Block):
    def __init__(self, vocab_len, embsize, nhidden, nlayers, nfc, nclass, drop_prob, **kwargs):
        super(EnhancedSeqInfer, self).__init__(**kwargs)
        with self.name_scope():
            # dropout prob
            self.drop_prob = drop_prob
            # word embedding
            self.embedding_layer = nn.Embedding(vocab_len, embsize)
            # first lstm, from sentence embed to hidden outputs
            self.bilstm1 = rnn.LSTM(nhidden, num_layers=nlayers, dropout=drop_prob, bidirectional=True)
            # second lstm, from augmented embed to m
            self.bilstm2 = rnn.LSTM(nhidden, num_layers=1, dropout=drop_prob, bidirectional=True)
            # enhancement
            self.align_att = AlignAttention()
            # this layer is used to output the final class
            self.output_layer = nn.HybridSequential()
            self.output_layer.add(nn.Dense(nfc, activation='tanh'), nn.Dropout(rate=drop_prob), \
                                  nn.Dense(nfc, activation='tanh'), nn.Dropout(rate=drop_prob), \
                                  nn.Dense(nclass))

    def forward(self, inp_left, inp_right):
        # inp is a list containing left_text and right_text
        # their size: [batch, token_idx]
        # inp_embed_left/right size: [batch, seq_len, embed_size]
        inp_embed_left = self.embedding_layer(inp_left)
        inp_embed_right = self.embedding_layer(inp_right)
        # rnn requires the first dimension to be the time steps, output is (seq_len, batch_size, embed_size)
        h_output_left = self.bilstm1(nd.transpose(inp_embed_left, axes=(1, 0, 2)))
        h_output_right = self.bilstm1(nd.transpose(inp_embed_right, axes=(1, 0, 2)))
        m_left, m_right = self.align_att(nd.transpose(h_output_left, axes=(1, 0, 2)), \
                                                      nd.transpose(h_output_right, axes=(1, 0, 2)))
        # apply another layer of lstm
        # v_left/right shape is (seq_len, batch_size, embed_size)
        v_left = self.bilstm2(nd.transpose(m_left, axes=(1, 0, 2)))
        v_right = self.bilstm2(nd.transpose(m_right, axes=(1, 0, 2)))
        # restore v's shape (batch_size, seq_len, embed_size)
        v_left = nd.transpose(v_left, axes=(1, 0, 2))
        v_right = nd.transpose(v_right, axes=(1, 0, 2))
        # apply max pooling 1D and avg pooling 1D
        v_left_avg = nd.sum(v_left, axis=1) / v_left.shape[1]
        v_right_avg = nd.sum(v_right, axis=1) / v_right.shape[1]
        v_left_max = nd.max(v_left, axis=1)
        v_right_max = nd.max(v_right, axis=1)
        # concatenate these 4 matrices
        dense_input = nd.concat(v_left_avg, v_left_max, v_right_avg, v_left_max, dim=-1)
        
        output = self.output_layer(dense_input)
        return output

## Configure parameters and build models

The resulting `M` is a matrix, and the way to classify this matrix is `flatten`, `mean` or `prune`. Prune is a way of trimming parameters proposed in the original paper and has been implemented here.

In [ ]:
vocab_len = len(vocab)
emsize = 200   # word embedding size
nhidden = 300    # lstm hidden_dim
nlayers = 3     # lstm layers

# final fc layer's number of hidden units and predicted number of classes
nfc = 1024
nclass = 2

drop_prob = 0

ctx = try_gpu()

model = EnhancedSeqInfer(vocab_len, emsize, nhidden, nlayers, nfc, nclass, drop_prob)

model.initialize(init=init.Xavier(), ctx=ctx)

# Attach a pre-trained glove word vector to the embedding layer
model.embedding_layer.weight.set_data(vocab.embedding.idx_to_vec)
# fixed the embedding layer
model.embedding_layer.collect_params().setattr('grad_req', 'null')

print(model)

train_curve, valid_curve = [], []

In [ ]:
def calculate_loss(x_left, x_right, y, model, loss, class_weight):
    pred = model(x_left, x_right)
    y = nd.array(y.asnumpy().astype('int32')).as_in_context(ctx)
    if loss_name in ['sce', 'l1', 'l2']:
        l = loss(pred, y)
    elif loss_name == 'wsce':
        l = loss(pred, y, class_weight, class_weight.shape[0])
    else:
        raise NotImplemented
    return pred, l

In [ ]:
def one_epoch(data_iter, model, loss, trainer, ctx, is_train, epoch, \
              clip=None, class_weight=None, loss_name='sce'):

    loss_val = 0.
    total_pred = []
    total_true = []
    n_batch = 0

    for batch_x_left, batch_x_right, batch_y in data_iter:
        batch_x_left = batch_x_left.as_in_context(ctx)
        batch_x_right = batch_x_right.as_in_context(ctx)
        batch_y = batch_y.as_in_context(ctx)

        if is_train:
            with autograd.record():
                batch_pred, l = calculate_loss(batch_x_left, batch_x_right, \
                                               batch_y, model, loss, class_weight)

            # backward calculate
            l.backward()

            # clip gradient
            clip_params = [p.data() for p in model.collect_params().values()]
            if clip is not None:
                norm = nd.array([0.0], ctx)
                for param in clip_params:
                    if param.grad is not None:
                        norm += (param.grad ** 2).sum()
                norm = norm.sqrt().asscalar()
                if norm > clip:
                    for param in clip_params:
                        if param.grad is not None:
                            param.grad[:] *= clip / norm

            # update parmas
            trainer.step(batch_x_left.shape[0])

        else:
            batch_pred, l = calculate_loss(batch_x_left, batch_x_right, \
                                           batch_y, model, loss, class_weight)

        # keep result for metric
        batch_pred = nd.argmax(nd.softmax(batch_pred, axis=1), axis=1).asnumpy()
        batch_true = np.reshape(batch_y.asnumpy(), (-1, ))
        total_pred.extend(batch_pred.tolist())
        total_true.extend(batch_true.tolist())
        
        batch_loss = l.mean().asscalar()

        n_batch += 1
        loss_val += batch_loss

        # check the result of traing phase
        if is_train and n_batch % 400 == 0:
            print('epoch %d, batch %d, batch_train_loss %.4f, batch_train_acc %.3f' %
                  (epoch, n_batch, batch_loss, accuracy_score(batch_true, batch_pred)))

    # metric
    F1 = f1_score(np.array(total_true), np.array(total_pred), average='binary')
    acc = accuracy_score(np.array(total_true), np.array(total_pred))
    loss_val /= n_batch

    if is_train:
        print('epoch %d, learning_rate %.5f \n\t train_loss %.4f, acc_train %.3f, F1_train %.3f, ' %
              (epoch, trainer.learning_rate, loss_val, acc, F1))
        train_curve.append((acc, F1))
        # declay lr
        if epoch % 3 == 0:
            trainer.set_learning_rate(trainer.learning_rate * 0.9)
    else:
        print('\t valid_loss %.4f, acc_valid %.3f, F1_valid %.3f, ' % (loss_val, acc, F1))
        valid_curve.append((acc, F1))

In [ ]:
def train_valid(data_iter_train, data_iter_valid, model, loss, trainer, \
                ctx, nepochs, clip=None, class_weight=None, loss_name='sce'):

    for epoch in range(1, nepochs+1):
        start = time.time()
        # train
        is_train = True
        one_epoch(data_iter_train, model, loss, trainer, ctx, is_train,
                  epoch, clip, class_weight, loss_name)

        # valid
        is_train = False
        one_epoch(data_iter_valid, model, loss, trainer, ctx, is_train,
                  epoch, clip, class_weight, loss_name)
        end = time.time()
        print('time %.2f sec' % (end-start))
        print("*"*100)

## Train
Now that we are training the model, we use WeightedSoftmaxCE to alleviate the problem of data category imbalance.

In [ ]:
class_weight = None
loss_name = 'sce'
optim = 'adam'
lr = 0.001
clip = .5
nepochs = 10

trainer = gluon.Trainer(model.collect_params(), optim, {'learning_rate': lr})

if loss_name == 'sce':
    loss = gluon.loss.SoftmaxCrossEntropyLoss()
elif loss_name == 'wsce':
    loss = WeightedSoftmaxCE()
    # the value of class_weight is obtained by counting data in advance. It can be seen as a hyperparameter.
    class_weight = nd.array([1., 3.], ctx=ctx)
elif loss_name == 'l1':
    loss = gluon.loss.L1Loss()
elif loss_name == 'l2':
    loss = gluon.loss.L2Loss()

In [ ]:
# train and valid
train_valid(train_dataloader, valid_dataloader, model, loss, \
            trainer, ctx, nepochs, clip=clip, class_weight=class_weight, loss_name=loss_name)

## Predict
Now we will randomly input a movie plot summary and its review to see if the model decide the review is a spoiler or not.

In [38]:

'''
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = gluon.nn.SymbolBlock.imports("model/att-symbol.json", ['data'], \
                                         "model/att-0001.params", ctx=ctx)
'''

AssertionError: Parameter 'embedding0_weight' is missing in file 'model/att-0001.params', which contains parameters: 'selfattentivebilstm2_selfattentivebilstm12_embedding0_weight', 'selfattentivebilstm2_selfattentivebilstm12_lstm0_l0_i2h_weight', 'selfattentivebilstm2_selfattentivebilstm12_lstm0_l0_h2h_weight', ..., 'selfattentivebilstm2_selfattentivebilstm12_selfattention0_dense1_weight', 'selfattentivebilstm2_selfattentivebilstm12_selfattention0_dense1_bias', 'selfattentivebilstm2_selfattentivebilstm12_dense0_weight', 'selfattentivebilstm2_selfattentivebilstm12_dense0_bias'. Please make sure source and target networks have the same prefix.

In [57]:
right = 'john is a former police, his daughter, sarah was killed in a car accident, which he \
        did not think was purely accidental. with his investigation went deeper, he found \
        that sarah\'s boyfriend, jack, was the one to blame.'
left = 'Word embedding can effectively represent the semantic similarity between words, \
        which brings many breakthroughs for natural language processing tasks. \
        The attention mechanism can intuitively grasp the important semantic features \
        in the sentence. I liked sarah and jack in this movie, but hate john, because he \
        killed sarah'
left_token = vocab[tokenizer(left)]
right_token = vocab[tokenizer(right)]

In [58]:
left_input = nd.array(left_token, ctx=ctx).reshape(1,-1)
right_input = nd.array(right_token, ctx=ctx).reshape(1,-1)
pred = model(left_input, right_input)
pred
#model.embedding_layer_right.weight.list_data()


[[1.000000e+00 6.869653e-13]]
<NDArray 1x2 @gpu(0)>

In order to intuitively feel the role of the attention mechanism, we visualize the output of the model's attention on the predicted samples.